In [1]:
import copy
import re
from collections import defaultdict
import pickle

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Topics

In [2]:
topics = pd.read_csv("../topic_words.csv.zip")
topics.head(30)

,meta,topic,weight,type,word
0,env,Экология в целом,5,0,воздействие
1,env,Экология в целом,5,0,окружающая среда
2,env,Экология в целом,5,0,природа
3,env,Экология в целом,5,0,природоохранный
4,env,Экология в целом,5,0,экологический
5,env,Экология в целом,5,0,экологичный
6,env,Экология в целом,1,0,влияние
7,env,Экология в целом,1,0,документ
8,env,Экология в целом,1,0,документация
9,env,Экология в целом,1,0,защита


In [3]:
topics["topic"].value_counts()
pass

In [4]:
re_dict = {}
re_advanced_dict = {}
sheets_dict = {}
sheets_advanced_dict = {}
base_dict = {}
advanced_dict = {}
importance_dict = {}

for idx, row in topics.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        sheets_advanced_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
        advanced_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    if row["type"] == 1:
        sheets_advanced_dict[row["topic"]][row["word"]] = row["weight"]
        advanced_dict[row["topic"]][row["word"]] = 0
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

In [6]:
for key, val in sheets_advanced_dict.items():
    re_advanced_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

# Expert

In [7]:
regex_expert = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [8]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [9]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [10]:
df = pd.read_excel("../topics/paragraph_max_values_50.xlsx", sheet_name="paragraph_max_values_50", index_col=0).rename(
    columns={"Экспертный топик Вера": "true_expert"}
)
df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text,max_topic_cos,max_score_cos,second_max_topic_cos,second_max_score_cos,true_expert,Экспертный топик МА,Комментарий
2023915.0,ПАО «Газпром»,"2015, 2014",Энергетика,ОУР,1942.0,Компании Группы Газпром вносят свой вклад в ре...,компания группа газпром вносить свой вклад реа...,Биоразнообразие,0.678372,NaN,NaN,Биоразнообразие,NaN,NaN
1515679.0,ПАО «Газпром»,2011.0,Энергетика,ЭО,761.0,"пересажено более 3 000 редких растений, в том ...",пересадить редкий растение число реликтовый ви...,Биоразнообразие,0.657152,NaN,NaN,Биоразнообразие,NaN,NaN
699878.0,Госкорпорация «Росатом»,2018.0,Энергетика,ИО,1288.0,Вклад в сохранение биоразнообразия мощности до...,активность радионуклид поступить сточный вода ...,Биоразнообразие,0.647845,NaN,NaN,Биоразнообразие,NaN,NaN
699878.0,Госкорпорация «Росатом»,2018.0,Энергетика,ИО,1288.0,"_x000c_Активность бета-активных радионуклидов,...",активность радионуклид поступить сточный вода ...,Биоразнообразие,0.647845,NaN,NaN,NaN,NaN,"сложный кейс, безопасность населения"
1383089.0,ПАО «Газпром»,2014.0,Энергетика,ЭО,1165.0,"«Белоплечий орлан», «Птицы Сахалина», «Реки Са...",белоплечий орлан птица сахалин река сахалин ки...,Биоразнообразие,0.647845,NaN,NaN,Биоразнообразие,NaN,NaN


In [11]:
vector = []
labels = []
found_words = []
true_expert_found_words = []
texts = []

for i, (text, expert_topic, original_text) in tqdm(
    enumerate(zip(df["cleaned_text"], df["true_expert"], df["original_text"])), total=df["cleaned_text"].shape[0]
):
    tmp = np.zeros(topics_vectors.shape[1])
    if pd.isna(text):
        continue

    if not re_dict.get(expert_topic, False):
        continue

    if len(re_dict[expert_topic].findall(text)) < 5:
        continue

    texts.append({"text": text, "topic": expert_topic, "original_text": original_text})

    text_words = {}
    for key, regex in re_dict.items():
        words = regex.findall(text)
        text_words[key] = words
        if key == expert_topic:
            true_expert_found_words.append((key, words))
        for word in words:
            tmp[word2idx[word]] = 1
    vector.append(tmp)
    labels.append(expert_topic)
    found_words.append(text_words)

100%|███████████████████████████████████████| 2038/2038 [02:32<00:00, 13.37it/s]


In [12]:
pd.DataFrame(texts).to_csv("used_texts_expert.csv")

In [13]:
vector = np.vstack(vector)

In [14]:
w = pd.DataFrame(found_words)
w.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,!Кибербезопасность,Права человека,Лидерство,Риски,Этика и антикоррупция,Корпоративное управление,Устойчивое развитие
0,"[природоохранный, воздействие, экологический, ...","[(реализация, ), (регион, ), (предотвращение, ...","[территория, вид, нефть, нефть, режим, работа,...","[негативный, воздействие, экологический, зона,...","[негативный, воздействие, морской, морской, зо...","[охрана, окружающий, среда, негативный, воздей...","[международный, животное, международный, приро...","[российский, российский, среда, негативный, во...","[негативный, воздействие, качество, негативный...","[отношение, работа, работа, работа, работа]",...,"[стандарт, отчётный, контроль, наблюдатель, ко...","[деятельность, отношение, проект, сообщество, ...",[],[безопасность],"[соблюдение, предотвращение, случай, предотвра...","[период, совет, государственный, консультативн...","[охрана, мероприятие, сохранение, влияние, без...",[уязвимый],"[проведение, проведение, ответственность, пров...","[вклад, деятельность, мероприятие, основа, дея..."
1,"[природа, экологический, природа, система, цел...","[(мероприятие, ), (территория, ), (проект, ), ...","[вид, работа, работа, вид, работа, общий, терр...","[экологический, экологический, солнечный]","[река, приток, качество, речной, вода, река, р...","[охрана, минимизация, использование, затрата, ...","[редкий, вид, животное, сохранение, миграция, ...","[растение, мероприятие, сохранение, ущерб, поч...","[качество, вода, использование, выращивание, и...","[работа, работа, работа, работа]",...,[],"[финансовый, деятельность, деятельность, проек...","[технология, уникальный]",[],"[человек, человек, национальный, национальный,...","[период, период, человек, человек, государстве...","[мероприятие, охрана, сохранение, ущерб, возни...",[],"[проведение, проведение, возможность, проведение]","[мероприятие, деятельность, деятельность, зада..."
2,"[влияние, контроль, влияние, природоохранный, ...","[(проект, ), (территория, ), (проект, ), (иниц...","[мощность, мощность, вид, территория, инфрастр...","[зона, зона, управление, зона, управление, ком...","[сточный, вода, поверхностный, водный, зона, з...","[окружающий, среда, инициатива, инициатива, пр...","[сохранение, сохранение, редкий, исчезать, вид...","[поверхностный, объект, среда, контроль, сохра...","[вода, оценка, содержание, продукт, продукт, п...","[работа, работа, доля]",...,"[контроль, рамка, рамка, порядок, контроль, не...","[проект, проект, проект]","[инициатива, инициатива, технология]",[],"[национальный, национальный]","[государственный, независимый, государственный]","[влияние, сохранение, сохранение, влияние, сох...",[],[],"[вклад, инициатива, дополнительный, инициатива..."
3,"[экологический, природа, экологический, эколог...","[(цель, ), (воздействие, ), (рост, ), (меропри...","[нефть, нефть, работа, нефть, нефть, работа, с...","[экологический, экологический, нефть, нефть, у...","[река, воздействие, морской, учёт, воздействие...","[электронный, экологический, экологический, эк...","[международный, сохранение, экологический, сох...","[сохранение, экологический, сохранение, сохран...","[воздействие, оценка, воздействие]","[отношение, работа, работа, работа, работа]",...,"[рамка, рамка, порядок, отчёт]","[отношение, финансовый, акция]",[],[],"[человек, пол]","[человек, представитель, представитель, госуда...","[инвестировать, план, сохранение, план, сохран...",[проблема],[],"[мероприятие, отчёт]"
4,"[защита, экологический, природа, экологический...","[(территория, ), (мониторинг, ), (реализация, ...","[работа, территория, область, система, террито...","[экологический, экологический, экологический, ...","[водный, озеро, река, очистка, озеро, озеро, о...","[защита, экологический, экологический, экологи...","[сохранение, охранять, природный, восстан

In [15]:
p = pd.DataFrame(true_expert_found_words).rename(columns={0: "topic", 1: "words"})
p.head()

,topic,words
0,Биоразнообразие,"[международный, животное, международный, приро..."
1,Биоразнообразие,"[редкий, вид, животное, сохранение, миграция, ..."
2,Биоразнообразие,"[сохранение, сохранение, редкий, исчезать, вид..."
3,Биоразнообразие,"[международный, сохранение, экологический, сох..."
4,Биоразнообразие,"[сохранение, охранять, природный, восстановлен..."


In [16]:
print(f"{'Topic':<35} {'Average Word Length':<20} Count")
for topic, val in p.groupby("topic")["words"].apply(list).items():
    print(f"{topic:<35} {sum([len(e) for e in val]) / len(val):05.2f} {len(val)}")

Topic                               Average Word Length  Count
Биоразнообразие                     32.27 52
Вода                                51.56 43
Воздух                              27.96 28
Заинтересованные стороны            16.00 3
Закупки и антикоррупция             30.55 42
Климат                              25.58 48
Корпоративное управление            12.79 19
Лидерство                           25.00 2
Малый и локальный бизнес            08.93 28
Обучение и развитие                 46.17 53
Оплата труда                        24.45 56
Отношения с инвесторами             25.97 33
Отчетность и прозрачность           12.80 15
Поставщики в целом                  18.21 53
Поставщики. Работники               10.18 11
Поставщики. Экология                09.25 16
Потребители. Доступность            11.92 25
Потребители. Здоровье и благопо     13.07 28
Потребители. Качество и безопас     27.34 70
Потребители. Маркетинг, продажи     13.64 45
Потребители. Персональные данны     20.

In [17]:
def init_matrix(topics: pd.DataFrame) -> np.ndarray:
    matrix = np.zeros(topics_vectors.shape)
    # topic	weight	type	word
    for idx, row in topics.iterrows():
        matrix[label2idx[row["topic"]], word2idx[row["word"]]] = 1
    return matrix


def fit(matrix, vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if not label2idx.get(label, False):
                continue
            # print(label)
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector
    return matrix


def test(matrix, vectors, labels, debug=False):
    total = 0
    pred_labels = []
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        # print(np.argmax(vector @ matrix.T))
        total += label == idx2label[res]
        pred_labels.append(idx2label[res])
        if debug and label != idx2label[res]:
            print("expected", label, "result", idx2label[res], vector @ matrix.T)
    return pred_labels, total / len(labels)

In [18]:
matrix = init_matrix(topics)

In [19]:
for i in tqdm(range(10)):
    matrix = fit(matrix, vector, labels)
    # test(vector, labels, False)

pred_labels, acc = test(matrix, vector, labels, False)
acc

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.97it/s]


0.46906474820143884

In [20]:
matrix_df = pd.DataFrame(matrix, columns=topics["word"].tolist(), index=topics["topic"].unique().tolist())
matrix_df.head()

,воздействие,окружающая среда,природа,природоохранный,экологический,экологичный,влияние,документ,документация,защита,...,соответствовать,управлять,фокусный,формировать,iso 14001,science based targets initiative sbti,верификация,целевый показатель,система экологического менеджмент,сэм
Экология в целом,-2650.0,0.0,-480.0,-1170.0,-3030.0,-120.0,-1220.0,-1770.0,-460.0,-1720.0,...,0.0,-270.0,-10.0,-270.0,0.0,0.0,-20.0,0.0,0.0,-100.0
Климат,-2650.0,0.0,-480.0,-1170.0,-3030.0,-120.0,-1220.0,-1770.0,-460.0,-1720.0,...,0.0,-270.0,-10.0,-270.0,0.0,0.0,-20.0,0.0,0.0,-100.0
Энергия,-2650.0,0.0,-480.0,-1170.0,-3030.0,-120.0,-1220.0,-1770.0,-460.0,-1720.0,...,0.0,-270.0,-10.0,-270.0,0.0,0.0,-20.0,0.0,0.0,-100.0
Воздух,-2229.0,0.0,21.0,-669.0,-2429.0,-120.0,-760.0,-1730.0,-440.0,-1520.0,...,0.0,-270.0,-10.0,-270.0,0.0,0.0,-20.0,0.0,0.0,-100.0
Вода,-2069.0,0.0,-480.0,-1030.0,-2730.0,-120.0,-1000.0,-1630.0,-460.0,-1680.0,...,0.0,-270.0,-10.0,-270.0,0.0,0.0,-20.0,0.0,0.0,-80.0


In [21]:
np.save("expert.npy", matrix)
matrix_df.to_csv("expert_matrix.csv")
matrix_df.T.to_csv("expert_matrix.T.csv")

In [22]:
results = []
for col in tqdm(matrix_df.columns):
    max_val = matrix_df[col].max()[0] if type(matrix_df[col].max()) is not float else matrix_df[col].max()
    min_val = matrix_df[col].min()[0] if type(matrix_df[col].min()) is not float else matrix_df[col].min()
    if max_val > 0:
        results.append((col, max_val))
    if min_val < -2000:
        results.append((col, min_val))

results_df = pd.DataFrame(results, columns=["word", "val"])
results_df.to_csv("matrix_words.csv")

100%|██████████████████████████████████████| 3650/3650 [00:05<00:00, 646.73it/s]


In [23]:
# calculate f1, prescion, recall
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print(
    "accuracy",
    accuracy_score(
        labels,
        pred_labels,
    ),
)
print(
    "f1",
    f1_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "precision",
    precision_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "recall",
    recall_score(
        labels,
        pred_labels,
        average="macro",
    ),
)

accuracy 0.46906474820143884
f1 0.29415071880397875
precision 0.47659766833474276
recall 0.3057101728583591


/Users/darlinkennedy/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# fit(topics_vectors, list(sorted(topics["topic"].unique())))
# test(topics_vectors, list(sorted(topics["topic"].unique())))

In [24]:
topics_vectors.shape

(45, 3650)

In [107]:
df = pd.read_csv("paragraphs_2022.csv")  # , usecols=["cleaned_text"], nrows=ROWS
df = df.dropna().reset_index(drop=True)
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

(8405, 8)

In [108]:
text = df.loc[100, "cleaned_text"]

In [109]:
text

'наш нераспределённый стоимость 2022 год составить 18,2 млрд руб. увеличиться 152,2 сравнение предыдущий год данный изменение обусловить положительный динамика показатель ebitda число завершение продажа силовой остров снижение ряд расход совместно решение выплачивать дивиденд 2022 год привести рост нераспределённый прибыль'

In [42]:
from typing import Optional
from typing import Union
import typer

In [110]:
text = df.loc[100, "cleaned_text"]
# text = "Доля сотрудников, охваченных коллективными договорами".lower()


def test_algos(text: str, verbouse: bool = False) -> dict[str, Union[str, float, list[str],None]]:
    arr = []
    result = {"text": text}
    vector = np.zeros(matrix.shape[1])

    for key, regex in re_dict.items():
        words = regex.findall(text)
        paragraph_dict = copy.deepcopy(base_dict)
        #print(paragraph_dict)
        importance = 0
        paragraph_len = len(text.split())
        base_words = 0
        for word in words:
            if type(word) is tuple:
                print(word)
                word = word[0]
            paragraph_dict[key][word] = sheets_dict[key][word]
            importance += sheets_dict[key][word] > 1
            vector[word2idx[word]] = 1
            base_words += 1
        # if base_words / paragraph_len < 0.1 or importance < 1:
        #     arr.append((key, 0, 0, []))
        #     continue

        dist = cosine_similarity(X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())])[0][0]
        arr.append((copy.deepcopy(key), copy.deepcopy(dist), copy.deepcopy(importance), copy.deepcopy(words)))
        result = {**result, **{f"{key}_dist": dist, f"{key}_importance": importance, f"{key}_words": words}}
    if verbouse:
        print("\nexpert")
    res = vector @ matrix.T
    for idx in np.argsort(res)[::-1]:
        if verbouse:
            print(idx2label[idx], res[idx])
        result[f"{idx2label[idx]}_expert"] = res[idx]

    if verbouse:
        print("\ncos")
        for key, dist, importance, words in sorted(arr, key=lambda x: x[1], reverse=True):
            print(f"{key} расстояние {dist:.4f} важные слова {importance} {copy.deepcopy(set(words))}")
    return result

In [111]:
text = df.loc[100, "cleaned_text"]
print(text)
test_algos(text, True)
a = 0

наш нераспределённый стоимость 2022 год составить 18,2 млрд руб. увеличиться 152,2 сравнение предыдущий год данный изменение обусловить положительный динамика показатель ebitda число завершение продажа силовой остров снижение ряд расход совместно решение выплачивать дивиденд 2022 год привести рост нераспределённый прибыль
('показатель', '')
('снижение', '')
('рост', '')

expert
Экология в целом -31150.0
Энергия -31711.0
Климат -34237.0
Вода -34361.0
Сотрудники. Безопасность и охра -34670.0
Сотрудники. Вовлеченность и мот -35830.0
Оплата труда -36090.0
Социальные инвестиции и благотв -36168.0
Потребители. Удовлетворенность -36204.0
Потребители. Ценовая политика -36298.0
Воздух -36350.0
Потребители. Сервис и коммуника -36493.0
Потребители. Качество и безопас -36745.0
Обучение и развитие -36890.0
Отношения с инвесторами -36987.0
Сотрудники. Профсоюз и Коллекти -37010.0
Закупки и антикоррупция -37068.0
Биоразнообразие -37210.0
Корпоративное управление -37370.0
Рекультивация земель -37446.0

In [44]:
def get_scores(text: str) -> dict[str, Union[str, float, list[str],None]]:
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    base_words = {}
    advanced_words = {}
    vector_exp = np.zeros(matrix.shape[1])
    if type(text) is float:
        print(text, type(text))
        return {}
    paragraph_len = len(text.split())

    total_found = 0
    for key, regex in re_dict.items():
        words = regex.findall(text)
        # advanced_words_found = re_advanced_dict[key].findall(text)
        for word in words:
            found_words = True
            if type(word) is tuple:
                word = word[0]
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            paragraph_importance[key] += weight > 1

            vector_exp[word2idx[word]] = 1

        base_words[key] = len(words)
        total_found += len(words)
        # advanced_words[key] = len(advanced_words_found)

    debug_dict = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        debug_dict["total_found_words"] = total_found
        return {**res, **{"max_score_cos": None, "max_topic_cos": None, "count_words": None}, **debug_dict}

    topic_found = False
    res = {}
    for key, vector in sheets_dict.items():
        debug_dict[f"{key}_percent_of_words"] = base_words[key] / paragraph_len
        debug_dict[f"{key}_importance"] = paragraph_importance[key]
        debug_dict[f"{key}_base_words_count"] = base_words[key]
        if base_words[key] / paragraph_len < 0.1 or paragraph_importance[key] < 1:
            res[key] = 0
            continue
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
        topic_found = True

    if not topic_found:
        res = {key: 0 for key in sheets_dict}
        return {**res, **{"max_score_cos": -100, "max_topic_cos": "безтемы", "count_words": None}, **debug_dict}

    paragraph_matrix = vector_exp @ matrix.T
    best_indicies = paragraph_matrix.argsort()[-2:][::-1]
    best_topics = [idx2label[i] for i in best_indicies]
    best_scores = paragraph_matrix[best_indicies]

    res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
    max_value = res_sorted[0]
    second_values = res_sorted[1]
    if second_values[1] == 0:
        second_values = (None, None)

    return {
        **res,
        **{
            "max_topic_cos": max_value[0],
            "max_score_cos": max_value[1],
            "second_max_topic_cos": second_values[0],
            "second_max_score_cos": second_values[1],
            "max_topic_exp": best_topics[0],
            "max_score_exp": best_scores[0],
            "second_max_topic_exp": best_topics[1],
            "second_max_score_exp": best_scores[1],
            "total_found_words": total_found,
        },
        **debug_dict,
    }

In [112]:
topics = []

i = 0
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row["cleaned_text"]
    res = get_scores(text)
    res["idx"] = idx
    if res.get("max_score_cos") is None:
        # topics.append(res)
        continue
    res["cleaned_text"] = text
    # print(res["max_score_cos"], res["max_topic_cos"])
    topics.append(res)

100%|███████████████████████████████████████| 8405/8405 [04:59<00:00, 28.06it/s]


In [113]:
with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [114]:
with open("topics.pkl", "rb") as f:
    topics = pickle.load(f)

In [115]:
topics = pd.DataFrame(topics).set_index(keys="idx")

In [116]:
topics.to_csv("all_topics_2022.csv")

In [117]:
topics.to_csv("all_topics_2022.csv.zip")

In [118]:
df.head()

,Unnamed: 0,rsspp_index,index,company,paragraph,original_text,cleaned_text,year
0,0,1428,1428,ЭЛ5-Энерго 2022,42,алибек айбекович тналин генеральный директор э...,алибек айбекович тналин генеральный директор у...,2020
1,1,1428,1428,ЭЛ5-Энерго 2022,43,"в завершение хочу еще раз отметить, что данные...",завершение хотеть отметить дать достижение явл...,2020
2,2,1428,1428,ЭЛ5-Энерго 2022,47,представляем вашему вниманию наш второй отчет ...,представлять ваш внимание наш второй отчёт уст...,2020
3,3,1428,1428,ЭЛ5-Энерго 2022,48,наша гордость в 2022 году достижение показател...,наш гордость 2022 год достижение показатель но...,2020
4,4,1428,1428,ЭЛ5-Энерго 2022,50,мы прилагаем большие усилия для развития и мот...,прилагать больший усилие развитие мотивация со...,2020


In [119]:
topics.head(10)

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_text,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words
idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0,0,алибек айбекович тналин генеральный директор у...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,2,2,завершение хотеть отметить дать достижение явл...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,1,3,представлять ваш внимание наш второй отчёт уст...,None,NaN,Обучение и развитие,-39223.0,Социальные инвестиции и благотв,-40520.0,26.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0,0,наш гордость 2022 год достижение показатель но...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0,1,прилагать больший усилие развитие мотивация со...,None,NaN,Сотрудники. Безопасность и охра,-136036.0,Экология в целом,-144720.0,57.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.19878,0.0,...,0,0,осознать ответственность делать возможный выпо...,Лидерство,0.233224,Экология в целом,-54120.0,Поставщики в целом,-54779.0,34.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0,1,2022 год мочь гордость отметить достижение важ...,None,NaN,Сотрудники. Профсоюз и Коллекти,-82947.0,Социальные инвестиции и благотв,-83865.0,37.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0,0,20232025 год предыдущий версия неоднократно пр...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,1,2,нарастить объём социальный инвестиция сравнени...,Обучение и развитие,0.245327,Социальные инвестиции и благотв,-55163.0,Сотрудники. Здоровье и благопол,-64784.0,42.0


In [120]:
df = df.join(topics, rsuffix="topics")

In [121]:
df.head(10)

,Unnamed: 0,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words
0,0,1428,1428,ЭЛ5-Энерго 2022,42,алибек айбекович тналин генеральный директор э...,алибек айбекович тналин генеральный директор у...,2020,0.0,0.0,...,0.0,0.0,алибек айбекович тналин генеральный директор у...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1428,1428,ЭЛ5-Энерго 2022,43,"в завершение хочу еще раз отметить, что данные...",завершение хотеть отметить дать достижение явл...,2020,0.0,0.0,...,2.0,2.0,завершение хотеть отметить дать достижение явл...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1428,1428,ЭЛ5-Энерго 2022,47,представляем вашему вниманию наш второй отчет ...,представлять ваш внимание наш второй отчёт уст...,2020,0.0,0.0,...,1.0,3.0,представлять ваш внимание наш второй отчёт уст...,None,NaN,Обучение и развитие,-39223.0,Социальные инвестиции и благотв,-40520.0,26.0
3,3,1428,1428,ЭЛ5-Энерго 2022,48,наша гордость в 2022 году достижение показател...,наш гордость 2022 год достижение показатель но...,2020,0.0,0.0,...,0.0,0.0,наш гордость 2022 год достижение показатель но...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1428,1428,ЭЛ5-Энерго 2022,50,мы прилагаем большие усилия для развития и мот...,прилагать больший усилие развитие мотивация со...,2020,0.0,0.0,...,0.0,1.0,прилагать больший усилие развитие мотивация со...,None,NaN,Сотрудники. Безопасность и охра,-136036.0,Экология в целом,-144720.0,57.0
5,5,1428,1428,ЭЛ5-Энерго 2022,51,мы осознаем свою ответственность и делаем все ...,осознать ответственность делать возможный выпо...,2020,0.0,0.0,...,0.0,0.0,осознать ответственность делать возможный выпо...,Лидерство,0.233224,Экология в целом,-54120.0,Поставщики в целом,-54779.0,34.0
6,6,1428,1428,ЭЛ5-Энерго 2022,52,2022 года мы можем с гордостью отметить достиж...,2022 год мочь гордость отметить достижение важ...,2020,0.0,0.0,...,0.0,1.0,2022 год мочь гордость отметить достижение важ...,None,NaN,Сотрудники. Профсоюз и Коллекти,-82947.0,Социальные инвестиции и благотв,-83865.0,37.0
7,7,1428,1428,ЭЛ5-Энерго 2022,53,"20232025 годов, их предыдущие версии неоднокра...",20232025 год предыдущий версия неоднократно пр...,2020,0.0,0.0,...,0.0,0.0,20232025 год предыдущий версия неоднократно пр...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1428,1428,ЭЛ5-Энерго 2022,55,мы не только нарастили объем социальных инвест...,нарастить объём социальный инвестиция сравнени...,2020,0.0,0.0,...,1.0,2.0,нарастить объём социальный инвестиция сравнени...,Обучение и развитие,0.245327,Социальные инвестиции и благотв,-55163.0,Сотрудники. Здоровье и благопол,-64784.0,42.0
9,9,1428,1428,ЭЛ5-Энерго 2022,62,принципы подготовки отчетности отчет об устойч...,принцип подготовка отчётность отчёт устойчивый...,2020,0.0,0.0,...,2.0,6.0,принцип подготовка отчётность отчёт устойчивый...,Обучение и развитие,0.254588,Экология в целом,-168476.0,Социальные инвестиции и благотв,-180072.0,92.0


In [122]:
df.to_csv("paragrapghs_with_topisc_2022.csv")
df.to_csv("paragrapghs_with_topisc_2022.csv.zip")

In [78]:
df = pd.read_csv("paragrapghs_with_topisc_2020.csv.zip", index_col=0)
df.head()

,Unnamed: 0.1,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Этика и антикоррупция_importance,Этика и антикоррупция_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,count_words
0,0,79,79,GV Gold,7,обращение председателя совета директоров обращ...,обращение председатель совет директор обращени...,2020,0.0,0.0,...,0.0,1.0,0.058824,2.0,2.0,0.235294,5.0,8.0,обращение председатель совет директор обращени...,NaN
1,1,79,79,GV Gold,11,ответственное водопользование управление отход...,ответственный водопользование управление отход...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.041667,0.0,1.0,ответственный водопользование управление отход...,NaN
2,2,79,79,GV Gold,14,подход к управлению персоналом численность и с...,подход управление персонал численность структу...,2020,0.0,0.0,...,0.0,2.0,0.000000,0.0,0.0,0.057143,0.0,2.0,подход управление персонал численность структу...,NaN
3,3,79,79,GV Gold,17,подход к управлению развитием регионов вклад в...,подход управление развитие регион вклад развит...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.125000,1.0,3.0,подход управление развитие регион вклад развит...,NaN
4,4,79,79,GV Gold,20,gv gold динамично развивающаяся российская зол...,gv gold динамично развивающийся российский зол...,2020,0.0,0.0,...,0.0,0.0,0.017857,1.0,1.0,0.017857,1.0,1.0,gv gold динамично развивающийся российский зол...,NaN


In [57]:
df.shape

(47562, 199)

In [80]:
df = df[df["max_topic_exp"].isna()]
df = df[df["original_text"].str.len() > 100]
df.shape

(14675, 199)

In [81]:
df2 = df[df["max_topic_exp"].isna()]

In [82]:
df2.head(10)

,Unnamed: 0.1,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Этика и антикоррупция_importance,Этика и антикоррупция_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,count_words
6,6,79,79,GV Gold,23,в условиях макроэкономических и продолжающихся...,условие макроэкономический продолжаться эпидем...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.047619,0.0,1.0,условие макроэкономический продолжаться эпидем...,NaN
18,18,79,79,GV Gold,56,пао высочайший gv gold далее также группа ком...,высокий gv gold далее также группа компания gv...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,высокий gv gold далее также группа компания gv...,NaN
19,19,79,79,GV Gold,59,"уникальный портфель активов, прочное финансово...",уникальный портфель актив прочный финансовый п...,2020,0.0,0.0,...,0.0,1.0,0.022727,0.0,1.0,0.000000,0.0,0.0,уникальный портфель актив прочный финансовый п...,NaN
21,21,79,79,GV Gold,62,во всех направлениях нашей деятельности мы при...,направление наш деятельность придерживаться кл...,2020,0.0,0.0,...,0.0,0.0,0.055556,0.0,2.0,0.083333,2.0,3.0,направление наш деятельность придерживаться кл...,NaN
22,22,79,79,GV Gold,63,наши ценности безопасность gv gold развивает д...,наш ценность безопасность gv gold развивать де...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.083333,1.0,2.0,наш ценность безопасность gv gold развивать де...,NaN
24,24,79,79,GV Gold,100,активы gv gold находятся на территории россии....,актив gv gold находиться территория россия кор...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.029412,0.0,1.0,актив gv gold находиться территория россия кор...,NaN
25,25,79,79,GV Gold,102,месторождение голец высочайший гок высочайший ...,месторождение голец высокий гок высокий включа...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,месторождение голец высокий гок высокий включа...,NaN
26,26,79,79,GV Gold,110,месторождение угахан угахан производственный ...,месторождение угахан угахан производственный а...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.025641,0.0,1.0,месторождение угахан угахан производственный а...,NaN
27,27,79,79,GV Gold,113,компания продолжает программу по расширению ми...,компания продолжать программа расширение минер...,2020,0.0,0.0,...,0.0,0.0,0.050000,0.0,1.0,0.000000,0.0,0.0,компания продолжать программа расширение минер...,NaN
29,29,79,79,GV Gold,115,иргиредмет приступила к проектированию будущег...,иргиредмет приступить проектирование будущее п...,2020,0.0,0.0,...,0.0,0.0,0.030303,1.0,1.0,0.030303,0.0,1.0,иргиредмет приступить проектирование будущее п...,NaN


In [83]:
col = ["company", "original_text", "cleaned_text"]
col.extend(df.columns[df.columns.str.contains("_base_words_count")].tolist())

In [84]:
df[col].sample(1_000).to_csv("paragrapghs_with_topisc_sample_2021.csv")

In [85]:
df.groupby("max_topic_cos").apply(lambda x: x.nlargest(30, "max_score_cos"))[
    [
        "company",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_topic_cos",
        "max_score_cos",
        "second_max_topic_cos",
        "second_max_score_cos",
        "max_topic_exp",
        "max_score_exp",
        "second_max_topic_exp",
        "second_max_score_exp",
    ]
].to_csv(
    "paragraph_max_values_30_2021.csv"
)  # .reset_index()

In [86]:
df

,Unnamed: 0.1,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Этика и антикоррупция_importance,Этика и антикоррупция_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,count_words
6,6,79,79,GV Gold,23,в условиях макроэкономических и продолжающихся...,условие макроэкономический продолжаться эпидем...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.047619,0.0,1.0,условие макроэкономический продолжаться эпидем...,NaN
18,18,79,79,GV Gold,56,пао высочайший gv gold далее также группа ком...,высокий gv gold далее также группа компания gv...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,высокий gv gold далее также группа компания gv...,NaN
19,19,79,79,GV Gold,59,"уникальный портфель активов, прочное финансово...",уникальный портфель актив прочный финансовый п...,2020,0.0,0.0,...,0.0,1.0,0.022727,0.0,1.0,0.000000,0.0,0.0,уникальный портфель актив прочный финансовый п...,NaN
21,21,79,79,GV Gold,62,во всех направлениях нашей деятельности мы при...,направление наш деятельность придерживаться кл...,2020,0.0,0.0,...,0.0,0.0,0.055556,0.0,2.0,0.083333,2.0,3.0,направление наш деятельность придерживаться кл...,NaN
22,22,79,79,GV Gold,63,наши ценности безопасность gv gold развивает д...,наш ценность безопасность gv gold развивать де...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.083333,1.0,2.0,наш ценность безопасность gv gold развивать де...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47502,47502,1134,1134,АО «Сетевая компания,3368,"1442 о закупках инновационной продукции, высок...",1442 закупка инновационный продукция высокотех...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1442 закупка инновационный продукция высокотех...,NaN
47506,47506,1134,1134,АО «Сетевая компания,3372,федеральный закон от 06.04.2011 83фз об электр...,федеральный закон 06.04.2011 83фз электронный ...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.023810,0.0,1.0,федеральный закон 06.04.2011 83фз электронный ...,NaN
47515,47515,1134,1134,АО «Сетевая компания,3387,"года, утвержденная распоряжением правительства...",год утвердить распоряжение правительство росси...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.016667,1.0,1.0,год утвердить распоряжение правительство росси...,NaN
47541,47541,1134,1134,АО «Сетевая компания,3581,объяснение существенности аспекта и границ под...,объяснение существенность аспект граница подхо...,2020,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.115385,0.0,3.0,объяснение существенность аспект граница подхо...,NaN


In [87]:
df.groupby("max_topic_exp").first()  # ["max_score"].first()

,Unnamed: 0.1,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Этика и антикоррупция_importance,Этика и антикоррупция_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,count_words
max_topic_exp,,,,,,,,,,,,,,,,,,,,,


In [66]:
df.to_csv("paragrapghs_with_topisc_2021.csv")

In [67]:
grouped = pd.DataFrame(df.groupby(["company", "max_topic_exp"])["cleaned_text"].agg(" ".join))
grouped.head()

,,cleaned_text
company,max_topic_exp,


In [68]:
e_topics = topics['meta'=='env']

KeyError: False

In [69]:
from collections import defaultdict

#topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics.iterrows(), total=topics.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

  0%|                                                 | 0/46213 [00:00<?, ?it/s]


KeyError: 'topic'

In [70]:
len_topics

defaultdict(dict, {})

In [216]:
from __future__ import annotations


def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        if sheets_dict[topic][word] not in count_words:
            count_words[sheets_dict[topic][word]] = 1
        else: count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][0]
    advanced_percent = count_words[2] / len_topics[topic][1] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [217]:
scores = []

for (company, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        if type(word) is tuple:
                word = word[0]
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|█████████████████████████████████████████| 395/395 [00:04<00:00, 92.02it/s]


In [219]:
grouped["score"] = scores
grouped.head(39)

cleaned_text  \
company           max_topic_exp                                                                        
HEINEKEN Ґ РЃбб®® Биоразнообразие                  мочь отслеживать фактор требовать газовый вид ...   
                  Вода                             повсеместный использование экологически чистый...   
                  Воздух                                     создание атмосфера творчество инновация   
                  Закупки и антикоррупция          рамка политика информирование нарушение получи...   
                  Климат                           5,5 снижение объём выброс парниковый газ 9,3 к...   
                  Корпоративное управление                         богуслав бишов директор маркетинг   
                  Обучение и развитие              вовлечение процесс заинтересованный сторона па...   
                  Оплата труда                     относительно уровень 2009 г относительно урове...   
                  Отношения с инвесторами          российский пивоваренный отрасль отчёт представ...   
                  Поставщики в целом               принцип деловой этика недавно анализ область п...   
                  Потребители. Здоровье и благопо  варить пиво создавать будущее будущее присоеди...   
                  Потребители. Качество и безопас  сертификат доверие работодатель который удосто...   
                  Потребители. Маркетинг, продажи    правило ответственный коммерческий коммуникация   
                  Потребители. Сервис и коммуника  сектор розничный торговля концерн heineken под...   
                  Потребители. Удовлетворенность                   15. оценка экономический социаль-   
                  Сотрудники. Безопасность и охра  улучшать условие работа жизнедеятельность hein...   
                  Сотрудники. Профсоюз и Коллекти  государственный трудовой инспекция компания he...   
                  Социальные инвестиции и благотв  устойчивый развитие компания heineken россия 2...   
                  Экология в целом                 успех международный концерн heineken n.v. долг...   
                  Энергия                          снижение удельный потребление энергия тепловой...   
PwC Ґ РЃбб®®      Биоразнообразие                  подготовить команда состав исследование аналит...   
                  Вода                             объём закупка бумага сравнение использование о...   
                  Закупки и антикоррупция          наш сообщение возможный нарушение внутренний г...   
                  Климат                           наш первый шаг успех давать мы уверенность соз...   
                  Обучение и развитие              концепция корпоративный ответственность предыд...   
                  Отношения с инвесторами          полтора год время выход первый отчёт произойти...   
                  Поставщики в целом               2 отчёт корпоративный ответственность отчёт ко...   
                  Поставщики. Экология                                            аудиторский палата   
                  Потребители. Здоровье и благопо                                    наш будущее наш   
                  Потребители. Качество и безопас  качество направление клиент рынок делать основ...   
                  Потребители. Сервис и коммуника  потенциал счёт разработка новый услуга отражат...   
                  Потребители. Удовлетворенность   бумага сберечь экспертный оценка 2 278 дерево ...   
                  Сотрудники. Вовлеченность и мот  рада результат глобальный опрос 2010 год 84 со...   
                  Сотрудники. Волонтерство         некоммерческий организация который сотрудничат...   
                  Сотрудники. Корпоративная культ  повышать ценность добиваться высокий старший с...   
                  Сотрудники. Профсоюз и Коллекти  который входить pwc некоммерческий партнёрство...   
                  Социальные инвестиции и благотв  27 проект реализовать область корпоративный фи...   
                  Экология в цел

In [220]:
grouped.to_csv("grouped_paragraphs_test1.csv.zip")